In [ ]:
url = "https://www.gao.gov/search?f%5B0%5D=content_type%3ABid%20Protest%20Decision&sort_by=docdate&sort_order=DESC&f%5B1%5D=date%3Astart%2B2000-01-01%2Bend%2B2020-01-01"

def extractor(url):
    response = requests.get(url)
    html = BeautifulSoup(response.text, "html.parser")
    details = html.find_all("div", class_ = "teaser-search--excerpt")
    decisions = html.find_all("div", class_ = "clearfix text-formatted field field--name-field-legal-prod-decision-status field--type-text-long field--label-hidden field__item")
    output = {'details' : details, 'decisions' : decisions}
    return(output)

def parser(dlist):
    details = dlist['details']
    decisions = dlist['decisions']
    
    results = pd.DataFrame()
    
    for d in range(0, len(details)):
        details_string = str(details[d])
        decisions_string = str(decisions[d])

        matter_pos = details_string.find("Matter of:")
        file_pos = details_string.find("File:")
        date_pos = details_string.find("Date:")
        end = details_string.find("</div>")
        matter = details_string[matter_pos:file_pos]
        file = details_string[file_pos:date_pos]
        date = details_string[date_pos:end]

        decision = decisions_string.split(">",1)[1]

        series = pd.Series([matter, file, date, decision])
        results = results.append(series, ignore_index = True)
    
    return(results)

In [ ]:
# Extract specific portions of the output
def highlight_extractor(x):
    # Cleans up the highlight paragraph
    new_string = str(x)
    first_pass = new_string.split("<!--HTML--><html><body><p>", 1)[1]
    second_pass = first_pass.split("</p>\n</body></html>\n", 1)[0]
    return(second_pass)

def decision_extractor(x, potential_patterns, patterns):
    if str(x).find("DECISION") != -1:
        for p in range(0, len(potential_patterns)):
            if str(x).find(potential_patterns[p]) != -1:
                patterns[0] = potential_patterns[p]
        return(general_splitter(x, patterns))
    else:
        return("No Decision")
    
def highlight_extractor(x):
    # Cleans up the highlight paragraph
    new_string = str(x)
    item_found = new_string.find("Highlights", 1)
    if item_found != -1:
        first_pass = new_string.split("<html><body><p>", 1)[1]
        second_pass = first_pass.split("</p>\n</body></html>", 1)[0]
        return(second_pass)
    elif item_found != -1:
        first_pass = new_string.split("<html><body><div>\n\t", 1)[1]
        second_pass = first_pass.split("</div>", 1)[0]
        return(second_pass)
    else:
        return("No Highlights")
    
def firm_name_extractor(x, patterns):
    item_found = str(x).find(patterns[0], 1)
    if item_found != -1:
        return(general_splitter(x, patterns))
    else:
        return("Check this entry")
    
def date_extractor(x, patterns):
    item_found = str(x).find(patterns[0], 1)
    if item_found != -1:
        return(general_splitter(x, patterns))
    elif item_found == "The decision issued on the date below was subject to a GAO Protective Order.":
        return("The decision issued on the date below was subject to a GAO Protective Order.")
    else:
        return("Check this entry")
    
def status_extractor(x, patterns):
    try:
        return(general_splitter(x, patterns))
    except:
        return("Check this entry")

In [ ]:
def decision_extractor(x, patterns, potential_patterns):
    new_patterns = pattern_sequencer(x, patterns, potential_patterns)
    if new_patterns[0] != str("No Matching Pattern"):
        if len(str(x)) > 5:
            return(general_splitter(x, new_patterns))
        else:
            return("Missing Decision")
    else:
        return("Bad Extraction")
    
def date_extractor2(x, patterns, potential_patterns):
    new_patterns = pattern_sequencer(x, patterns, potential_patterns)
    if new_patterns[0] != str("No Matching Pattern"):
        return(general_splitter(x, new_patterns))
    elif len(str(x)) < 10:
        return("Short Decision")
    else:
        return("Bad Extraction")